In [1]:
!pip install datasets
!pip install gensim==4.3.1
!pip install tqdm
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 10.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/cola

In [2]:
from datasets import load_dataset
import pandas as pd
import json
import os
import re
from google.colab import drive
from tqdm import tqdm
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import nltk
import torch
import math
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from scipy.sparse import hstack
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/AI")

Mounted at /content/drive


In [4]:
hc3_df = pd.read_json(path_or_buf="eli5_clean_combined.jsonl", lines=True)
hc3_df

,question,human_answers,chatgpt_answers,index,scraped_answers
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,NaN,NaN
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,NaN,NaN
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,NaN,NaN
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,NaN,NaN
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,NaN,NaN
...,...,...,...,...,...
34351,What is closed-loop verification?,[Let's pretend that it's purge night and you a...,NaN,NaN,[Closed-loop verification is a process used in...
34352,Why is mustard oil illegal to consume in the US?,[Erucic acid is poison so there is a blanket b...,NaN,NaN,[Mustard oil is not illegal to consume in the ...
34353,How space probes shoot pictures back at Earth,[The technology to transmit information has be...,NaN,NaN,[Space probes shoot pictures back at Earth usi...
34354,What exactly do international waters mean for ...,[Every ship has a country of registry (flag st...,NaN,NaN,"[International waters, also known as the high ..."


In [5]:
# shift the data in scraped_answers to chatgpt_answers
for index, row in hc3_df.iterrows():
    # check if chatgpt_answers is nan
    if pd.isnull(row["chatgpt_answers"]).all() if type(row["chatgpt_answers"]) == list else pd.isnull(row["chatgpt_answers"]):
      hc3_df.at[index,"chatgpt_answers"] = row['scraped_answers']

hc3_df

,question,human_answers,chatgpt_answers,index,scraped_answers
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,NaN,NaN
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,NaN,NaN
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,NaN,NaN
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,NaN,NaN
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,NaN,NaN
...,...,...,...,...,...
34351,What is closed-loop verification?,[Let's pretend that it's purge night and you a...,[Closed-loop verification is a process used in...,NaN,[Closed-loop verification is a process used in...
34352,Why is mustard oil illegal to consume in the US?,[Erucic acid is poison so there is a blanket b...,[Mustard oil is not illegal to consume in the ...,NaN,[Mustard oil is not illegal to consume in the ...
34353,How space probes shoot pictures back at Earth,[The technology to transmit information has be...,[Space probes shoot pictures back at Earth usi...,NaN,[Space probes shoot pictures back at Earth usi...
34354,What exactly do international waters mean for ...,[Every ship has a country of registry (flag st...,"[International waters, also known as the high ...",NaN,"[International waters, also known as the high ..."


In [6]:
# drop index col which is all NaN
processed_hc3_df = hc3_df.drop(["scraped_answers"], axis=1)
processed_hc3_df = processed_hc3_df.drop(["index"], axis=1)
processed_hc3_df.dropna(inplace=True)
processed_hc3_df

,question,human_answers,chatgpt_answers
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...
...,...,...,...
34351,What is closed-loop verification?,[Let's pretend that it's purge night and you a...,[Closed-loop verification is a process used in...
34352,Why is mustard oil illegal to consume in the US?,[Erucic acid is poison so there is a blanket b...,[Mustard oil is not illegal to consume in the ...
34353,How space probes shoot pictures back at Earth,[The technology to transmit information has be...,[Space probes shoot pictures back at Earth usi...
34354,What exactly do international waters mean for ...,[Every ship has a country of registry (flag st...,"[International waters, also known as the high ..."


# Clean Dataset

In [7]:
# keep rows with non empty chatgpt ans
has_chatgpt_ans = processed_hc3_df["chatgpt_answers"].map(lambda ans_lst:ans_lst!=[])
processed_hc3_df = processed_hc3_df[has_chatgpt_ans]

#Need to expand the answers and chatgpt ans -- as for one question, there can be several human answers obtained
row_list = []
for index, row in processed_hc3_df.iterrows():
  human_ans_list = row["human_answers"]
  chatgpt_ans_list = row["chatgpt_answers"]
  # note that sometimes there can be 2 human ans, and 1 chatgpt ans for e.g. 
  num_chatgpt_ans = len(chatgpt_ans_list)
  for i in range(len(human_ans_list)):
    human_ans = human_ans_list[i]
    if i >= num_chatgpt_ans:
      chatgpt_ans = chatgpt_ans_list[num_chatgpt_ans-1]
    else:
      chatgpt_ans = chatgpt_ans_list[i]
    row_list.append([row["question"],human_ans,chatgpt_ans])

processed_hc3_df = pd.DataFrame(row_list, columns = ["question", "human_answer", "chatgpt_answer"])
print(processed_hc3_df.shape)
processed_hc3_df.head()

(68701, 3)


,question,human_answer,chatgpt_answer
0,"Why is every book I hear about a "" NY Times # ...","Basically there are many categories of "" Best ...",There are many different best seller lists tha...
1,"Why is every book I hear about a "" NY Times # ...","If you 're hearing about it , it 's because it...",There are many different best seller lists tha...
2,"Why is every book I hear about a "" NY Times # ...","One reason is lots of catagories . However , h...",There are many different best seller lists tha...
3,"If salt is so bad for cars , why do we use it ...",salt is good for not dying in car crashes and ...,Salt is used on roads to help melt ice and sno...
4,"If salt is so bad for cars , why do we use it ...","In Minnesota and North Dakota , they tend to u...",Salt is used on roads to help melt ice and sno...


# Convert the shape of the df to n x 2, where the first column is the response and the second column is the label.

In [8]:
chatgpt_responses = processed_hc3_df.loc[:, ["chatgpt_answer"]]
chatgpt_responses["label"]=1
chatgpt_responses.rename(columns = {'chatgpt_answer':'response'}, inplace = True)
chatgpt_responses.drop_duplicates(inplace=True)
print(chatgpt_responses.shape)
chatgpt_responses.head()

(33366, 2)


,response,label
0,There are many different best seller lists tha...,1
3,Salt is used on roads to help melt ice and sno...,1
6,There are a few reasons why we still have SD (...,1
9,It is generally not acceptable or ethical to a...,1
12,After the Wright Brothers made the first power...,1


In [9]:
human_responses = processed_hc3_df.loc[:, ["human_answer"]]
human_responses["label"]=0
human_responses.rename(columns = {'human_answer':'response'}, inplace = True)
human_responses.drop_duplicates(inplace=True)
print(human_responses.shape)
human_responses.head()

(64250, 2)


,response,label
0,"Basically there are many categories of "" Best ...",0
1,"If you 're hearing about it , it 's because it...",0
2,"One reason is lots of catagories . However , h...",0
3,salt is good for not dying in car crashes and ...,0
4,"In Minnesota and North Dakota , they tend to u...",0


In [10]:
# sample the same number of human responses as chatgpt responses to make type equal in number
sampled_human_responses = human_responses.sample(n = chatgpt_responses.shape[0])
sampled_human_responses

,response,label
22974,Pyroloysis and carbonization are the result of...,0
60650,If they can fit the product at the source into...,0
318,"We have the technology for both , but the manu...",0
23510,1929 stock market crash ( The Great Depression...,0
45472,English has always borrowed lots of words from...,0
...,...,...
3282,Maybe this [ gif ] ( URL_1 ) can help you . Oh...,0
2276,You should read about [ Thomas Beatie ] ( URL_...,0
62558,"inflation, supply and demand, etc will cause p...",0
55717,When visible light comes into contact with an ...,0


In [11]:
Corpus = pd.concat([chatgpt_responses, sampled_human_responses], ignore_index=True)
print(Corpus.shape)
Corpus.head()

(66732, 2)


,response,label
0,There are many different best seller lists tha...,1
1,Salt is used on roads to help melt ice and sno...,1
2,There are a few reasons why we still have SD (...,1
3,It is generally not acceptable or ethical to a...,1
4,After the Wright Brothers made the first power...,1


In [12]:
# Step - a : Remove blank rows if any.
Corpus['response'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['response'] = [entry.lower() for entry in Corpus['response']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['response']= [word_tokenize(entry) for entry in Corpus['response']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in tqdm(enumerate(Corpus['response'])):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

66732it [35:41, 31.16it/s]


In [13]:
Corpus.head()

,response,label,text_final
0,"[there, are, many, different, best, seller, li...",1,"['many', 'different', 'best', 'seller', 'list'..."
1,"[salt, is, used, on, roads, to, help, melt, ic...",1,"['salt', 'use', 'road', 'help', 'melt', 'ice',..."
2,"[there, are, a, few, reasons, why, we, still, ...",1,"['reason', 'still', 'sd', 'standard', 'definit..."
3,"[it, is, generally, not, acceptable, or, ethic...",1,"['generally', 'acceptable', 'ethical', 'advoca..."
4,"[after, the, wright, brothers, made, the, firs...",1,"['wright', 'brother', 'make', 'first', 'powere..."


In [14]:
Corpus.dropna()
Corpus

,response,label,text_final
0,"[there, are, many, different, best, seller, li...",1,"['many', 'different', 'best', 'seller', 'list'..."
1,"[salt, is, used, on, roads, to, help, melt, ic...",1,"['salt', 'use', 'road', 'help', 'melt', 'ice',..."
2,"[there, are, a, few, reasons, why, we, still, ...",1,"['reason', 'still', 'sd', 'standard', 'definit..."
3,"[it, is, generally, not, acceptable, or, ethic...",1,"['generally', 'acceptable', 'ethical', 'advoca..."
4,"[after, the, wright, brothers, made, the, firs...",1,"['wright', 'brother', 'make', 'first', 'powere..."
...,...,...,...
66727,"[maybe, this, [, gif, ], (, url_1, ), can, hel...",0,"['maybe', 'gif', 'help', 'oh', 'one', 'well']"
66728,"[you, should, read, about, [, thomas, beatie, ...",0,"['read', 'thomas', 'beatie', 'successfully', '..."
66729,"[inflation, ,, supply, and, demand, ,, etc, wi...",0,"['inflation', 'supply', 'demand', 'etc', 'caus..."
66730,"[when, visible, light, comes, into, contact, w...",0,"['visible', 'light', 'come', 'contact', 'objec..."


# Save the processed dataset

In [15]:
Corpus.to_csv("processed_dataset.csv", index=False)